In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install gensim
!pip install xgboost


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
%cd "/content/drive/MyDrive/Colab Notebooks/2023/SBBD_toxic"
!pwd

/content/drive/MyDrive/Colab Notebooks/2023/SBBD_toxic
/content/drive/MyDrive/Colab Notebooks/2023/SBBD_toxic


In [4]:
#baixe GloVe 
#!wget http://143.107.183.175:22980/download.php?file=embeddings/glove/glove_s300.zip
#!unzip glove_s300.zip
#!wget http://143.107.183.175:22980/download.php?file=embeddings/glove/glove.6B.300d-pt.zip
#!unzip glove.6B.300d-pt.txt

!unzip '/content/drive/MyDrive/Colab Notebooks/2023/SBBD_toxic/glove.6B.300d.txt'

from gensim.models import KeyedVectors

#glove_file = "glove_s300.txt"
#glove_embeddings = KeyedVectors.load_word2vec_format(glove_file, binary=False)


Archive:  /content/drive/MyDrive/Colab Notebooks/2023/SBBD_toxic/glove.6B.300d.txt
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/drive/MyDrive/Colab Notebooks/2023/SBBD_toxic/glove.6B.300d.txt or
        /content/drive/MyDrive/Colab Notebooks/2023/SBBD_toxic/glove.6B.300d.txt.zip, and cannot find /content/drive/MyDrive/Colab Notebooks/2023/SBBD_toxic/glove.6B.300d.txt.ZIP, period.


In [5]:
import pandas as pd


arquivo_csv_treino = '/content/drive/MyDrive/Colab Notebooks/2023/SBBD_toxic/data/train_toldbr.csv'
arquivo_csv_teste = '/content/drive/MyDrive/Colab Notebooks/2023/SBBD_toxic/data/test_toldbr.csv'

dados_treino = pd.read_csv(arquivo_csv_treino)
dados_teste = pd.read_csv(arquivo_csv_teste)

train_df = pd.DataFrame({'text': dados_treino.iloc[:, 2], 'labels': dados_treino.iloc[:, 1]})
eval_df = pd.DataFrame({'text': dados_teste.iloc[:, 2], 'labels': dados_teste.iloc[:, 1]})

print(train_df.head())


                                                text  labels
0  Meu nivel de amizade com isis é ela ter meu in...     1.0
1  rt @user @user o cara adultera dados, que fora...     1.0
2  @user @user @user o cara só é simplesmente o m...     1.0
3  eu to chorando vei vsf e eu nem staneio izone ...     1.0
4  tem um do jack com a msm música e agr não sei ...     0.0


In [6]:
import pandas as pd
import numpy as np
import re
import string
from gensim.parsing.preprocessing import remove_stopwords
from gensim.models import KeyedVectors
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding, Input
from tensorflow.keras.optimizers import Adam
import xgboost as xgb

# Função de pré-processamento
def preprocess(text):
    text = text.lower()  # Transformar para minúsculas
    text = remove_stopwords(text)  # Remover stopwords usando Gensim
    text = re.sub(f'[{re.escape(string.punctuation)}]', '', text)  # Remover pontuação
    return text

# Aplicar pré-processamento nos conjuntos de dados
train_df['text'] = train_df['text'].apply(preprocess)
eval_df['text'] = eval_df['text'].apply(preprocess)

# Carregar word embeddings pré-treinados
#word_embeddings_file = '/content/drive/MyDrive/Colab Notebooks/2023/SBBD_toxic/glove.6B.300d.txt'
#word_embeddings = KeyedVectors.load_word2vec_format(word_embeddings_file, binary=False)
def convert_glove_to_word2vec(glove_file, output_file):
    with open(glove_file, 'r', encoding='utf-8') as f_in:
        lines = f_in.readlines()
        num_vectors = len(lines)
        vector_size = len(lines[0].split()) - 1

    with open(output_file, 'w', encoding='utf-8') as f_out:
        f_out.write(f'{num_vectors} {vector_size}\n')
        for line in lines:
            f_out.write(line)

glove_file = '/content/drive/MyDrive/Colab Notebooks/2023/SBBD_toxic/glove.6B.300d.txt'
word2vec_output_file = '/content/drive/MyDrive/Colab Notebooks/2023/SBBD_toxic/glove.6B.300d-word2vec.txt'

convert_glove_to_word2vec(glove_file, word2vec_output_file)
word_embeddings = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)


# Tokenização e padding
max_length = 280
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df['text'])
word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(train_df['text'])
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post', truncating='post')

eval_sequences = tokenizer.texts_to_sequences(eval_df['text'])
eval_padded = pad_sequences(eval_sequences, maxlen=max_length, padding='post', truncating='post')

# Preparar matriz de embeddings
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 300
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in word_index.items():
    if word in word_embeddings:
        embedding_vector = word_embeddings[word]
        embedding_matrix[i] = embedding_vector

# Definir a arquitetura do modelo LSTM
inputs = Input(shape=(max_length,))
x = Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], trainable=False)(inputs)
x = LSTM(50, return_sequences=True)(x)
x = Dropout(0.5)(x)
x = LSTM(50)(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation='sigmoid')(x)

model = Model(inputs=inputs, outputs=outputs)

# Compilar e treinar o modelo
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_padded, train_df['labels'], epochs=10, batch_size=128, validation_split=0.1)

# Avaliar o modelo LSTM
eval_preds = model.predict(eval_padded)
eval_preds_class = np.where(eval_preds > 0.5, 1, 0)

# Calcular métricas de avaliação

f1 = f1_score(eval_df['labels'], eval_preds_class)
precision = precision_score(eval_df['labels'], eval_preds_class)
recall = recall_score(eval_df['labels'], eval_preds_class)

print(f'F1 Score: {f1}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')

# Remover a última camada (softmax) do modelo para extrair características
feature_extractor = Model(inputs=model.input, outputs=model.layers[-2].output)

#Extrair características do conjunto de teste
train_features = feature_extractor.predict(train_padded)
eval_features = feature_extractor.predict(eval_padded)

#Treinar um classificador XGBoost com as características extraídas
xgb_classifier = xgb.XGBClassifier()
#xgb_classifier.fit(eval_features, eval_df['labels'])
xgb_classifier.fit(train_features, train_df['labels'])

#Avaliar o classificador XGBoost
xgb_preds = xgb_classifier.predict(eval_features)
xgb_f1 = f1_score(eval_df['labels'], xgb_preds)
xgb_precision = precision_score(eval_df['labels'], xgb_preds)
xgb_recall = recall_score(eval_df['labels'], xgb_preds)

print(f'XGBoost F1 Score: {xgb_f1}')
print(f'XGBoost Precision: {xgb_precision}')
print(f'XGBoost Recall: {xgb_recall}')


Epoch 1/10
118/118 [==============================] - 14s 33ms/step - loss: 0.6899 - accuracy: 0.5453 - val_loss: 0.6673 - val_accuracy: 0.6841
Epoch 2/10
118/118 [==============================] - 3s 24ms/step - loss: 0.6897 - accuracy: 0.5453 - val_loss: 0.6629 - val_accuracy: 0.6841
Epoch 3/10
118/118 [==============================] - 3s 25ms/step - loss: 0.6893 - accuracy: 0.5453 - val_loss: 0.6607 - val_accuracy: 0.6841
Epoch 4/10
118/118 [==============================] - 3s 25ms/step - loss: 0.6894 - accuracy: 0.5453 - val_loss: 0.6670 - val_accuracy: 0.6841
Epoch 5/10
118/118 [==============================] - 3s 24ms/step - loss: 0.6893 - accuracy: 0.5453 - val_loss: 0.6725 - val_accuracy: 0.6841
Epoch 6/10
118/118 [==============================] - 3s 24ms/step - loss: 0.6894 - accuracy: 0.5453 - val_loss: 0.6688 - val_accuracy: 0.6841
Epoch 7/10
118/118 [==============================] - 3s 24ms/step - loss: 0.6892 - accuracy: 0.5452 - val_loss: 0.6655 - val_accuracy: 0.684

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


F1 Score: 0.0
Precision: 0.0
Recall: 0.0
131/131 [==============================] - 1s 7ms/step
XGBoost F1 Score: 0.3908549953022236
XGBoost Precision: 0.46017699115044247
XGBoost Recall: 0.3396842678279804
